In [ ]:
#Libraries
import os
import pandas as pd
import numpy as np
import keras
import tensorflow
import sklearn
from matplotlib import pyplot as plt
import sys
import surfboard
import scipy.io.wavfile as wav
from surfboard.feature_extraction_multiprocessing import extract_features

In [ ]:
#Misc. adjustments
pd.set_option('max_colwidth', 200)

In [ ]:
#Listing files
filelist = os.listdir("Source/vox")

#Read files into pandas
source = pd.DataFrame(filelist)
#Rename file name column
source = source.rename(columns={0:'file'})
#Filter out DS files from macOS
source = source[source.file != ".DS_Store"]
#Reset index
source = source.reset_index(drop = True)

#Extraction of variables from filenames
speaker_id = []
condition = []
index = []
duration=[]

for i in range (0, len(source)):
    id = source["file"][i][0:2]
    con = source["file"][i][3]
    indx = source["file"][i][5:7]
    (source_rate, source_sig) = wav.read(os.path.join(sys.path[0], "Source/vox/") + source["file"][i])
    dur = len(source_sig) / float(source_rate)

    if id[0:1] == "0":
        speaker_id.append(id[1])
    else:
        speaker_id.append(id)
    
    if indx[0:1] == "0":
        index.append(indx[1])
    else:
        index.append(indx)
    
    condition.append(con)
    duration.append(dur)
    
#Adding variables to dataframe
source["id"] = speaker_id
source["condition"] = condition
source["indx"] = index
source["duration"] = duration

mean_duration = source["duration"].mean()
print(mean_duration)
std_duration = source["duration"].std()
print(std_duration)

source.file = os.path.join(sys.path[0], "Source/vox/") + source.file
source_files = source["file"].tolist()

features = surfboard.feature_extraction_multiprocessing.extract_features_from_paths(source_files, ["mfcc", 'f0_statistics', 'formants'], statistics_list=['mean', "std"], sample_rate=44100, num_proc=3)
source = pd.concat([source, features], axis=1, sort=False)

source.head()

In [ ]:
#Create Training & Valuation Sets
def create_sets(subjects, n_train, n_val, conditions):
    train_set = pd.DataFrame()
    val_set = pd.DataFrame()
    selection = subjects
    
    if isinstance(conditions[0], tuple):
        for i in selection:
            tmp_train = source[(source.id==str(i)) & (source.condition==conditions[0][0])]
            tmp_train2 = source[(source.id==str(i)) & (source.condition==conditions[0][1])]
            tmp_train = tmp_train.sample(int(n_train/2))
            ind = tmp_train["indx"].tolist()
            tmp_train2 = tmp_train2[tmp_train2["indx"].isin(ind) == False]
            tmp_train2 = tmp_train2.sample(int(n_train/2))
            tmp_train = tmp_train.append(tmp_train2)
            train_set = train_set.append(tmp_train)
            ind = tmp_train["indx"].tolist()
                        
            tmp_val = source[(source.id==str(i)) & (source.condition==conditions[1])]
            tmp_val2 = tmp_val[tmp_val["indx"].isin(ind) == False]
            val_set = val_set.append(tmp_val2.sample(n_val))
    else:
        for i in selection:
            tmp_train = source[(source.id==str(i)) & (source.condition==conditions[0])]
            tmp_train = tmp_train.sample(n_train)
            train_set = train_set.append(tmp_train)
            ind = tmp_train["indx"].tolist()
            
            if isinstance(conditions[1], tuple):
                tmp_val = source[(source.id==str(i)) & (source.condition==conditions[1][0])]
                tmp_val2 = source[(source.id==str(i)) & (source.condition==conditions[1][1])]
                tmp_val = tmp_val.sample(int(n_val/2))
                ind = tmp_val["indx"].tolist()
                tmp_val2 = tmp_val2[tmp_val2["indx"].isin(ind) == False]
                tmp_val2 = tmp_val2.sample(int(n_val/2))
                tmp_val = tmp_val.append(tmp_val2)
                val_set = val_set.append(tmp_val)
                #print(tmp_val["indx"])
            else:
                tmp_val = source[(source.id==str(i)) & (source.condition==conditions[1])]
                tmp_val2 = tmp_val[tmp_val["indx"].isin(ind) == False]
                val_set = val_set.append(tmp_val2.sample(n_val))


    train_set = train_set.reset_index()
    val_set = val_set.reset_index()

    X_train = train_set.drop(train_set.columns[[0, 1, 2, 3, 4, 5]], axis=1)
    X_val = val_set.drop(val_set.columns[[0, 1, 2, 3, 4, 5]], axis=1)
    
    np.array(X_train)
    np.array(X_val)
    
    return train_set, val_set, X_train, X_val

In [ ]:
#Pre-Processing
def processing(X_train, X_val, train_set, val_set):
    y_train = np.array(train_set["id"])
    y_val = np.array(val_set["id"])
    
    from sklearn.preprocessing import LabelEncoder
    from keras.utils.np_utils import to_categorical
    lb = LabelEncoder()
    y_train = to_categorical(lb.fit_transform(y_train))
    y_val = to_categorical(lb.fit_transform(y_val))
    
    from sklearn.preprocessing import StandardScaler
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_val = ss.transform(X_val)
    
    return y_train, y_val, X_train, X_val

In [ ]:
#Build Model
def build_model(n_speakers):
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Activation, Flatten
    from keras.callbacks import EarlyStopping
    model = Sequential()
    model.add(Dense(19, input_shape=((source.shape[1] - 5),), activation = 'relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(128, activation = 'relu'))
    #model.add(Dropout(0.25))
    model.add(Dense(128, activation = 'relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(n_speakers, activation = 'softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')
    
    return model, early_stop


In [ ]:
#Train Model
def train_model(model, early_stop, log=0, graph=0):
    history = model.fit(X_train, y_train, batch_size=10, epochs=75,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stop], verbose = log)
    
    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    
    if graph == 1:
        plt.figure(figsize=(12, 8))
        plt.plot(train_accuracy, label='Training Accuracy', color='#185fad')
        plt.plot(val_accuracy, label='Validation Accuracy', color='orange')
        plt.title('Training and Validation Accuracy by Epoch', fontsize = 25)
        plt.xlabel('Epoch', fontsize = 18)
        plt.ylabel('Categorical Crossentropy', fontsize = 18)
        plt.xticks(range(0,100,5), range(0,100,5))
        plt.legend(fontsize = 18);
    
    return val_accuracy

In [ ]:
#Parameters

#Which subjects should be used in the experiment
selection=(1,3,5,8,10,11,12,13,15,18)
#Number of training samples per subject
n_train = 10
#Number of validation samples per subject (set to use all remianing samples after training samples have been chosen)
n_val = 20-n_train
#Conditions to be run (first element of tuple is training set, second element is validation set, tuple as element means set will be split between specfified conditions)
#0 is non-variable, 2 is variable
conditions = [("0","0"),("0","2"),("2","2"),("2","0"),(("0","2"),"0"),(("0","2"),"2"),("0",("0","2")),("2", ("0","2"))]
#Labels for conditions used for plotting and CSV export
labels = ["NV-NV","NV-V","V-V","V-NV","NV+V-NV","NV+V-V","NV-NV+V","V-NV+V"]
#Number of iterations per condition (in other words, how many models will be trained per condition)
iterations = 1
#Number of speakers, set to default to length of selection list
n_speakers = len(selection)
#Option to activate keras logging while training
log = 0
#Option to enable graphs for each individual model (NOTE: The graphs will not appear until all models have been trained)
graph = 0

results2 = [ [] for _ in range(iterations) ]
avg_results= []
results_list = []
colours = ["orange", "blue", "green", "red", "purple","dodgerblue", "yellow", "black"]

#Iterate through conditions
for i in range(len(conditions)):
    #Reset result list used for plotting average curves
    results1 = []
    
    #Iterate through number of chosen iterations per condition
    for j in range(iterations):
        
        #Creating sets
        [train_set, val_set, X_train, X_val] = create_sets(selection, n_train, n_val, conditions[i])
        
        #Pre-Processing
        [y_train, y_val, X_train, X_val] = processing(X_train, X_val, train_set, val_set)

        #Build model
        [model, early_stop] = build_model(n_speakers)

        #Train
        result = train_model(model, early_stop, log, graph)
        results1.append(result)
        result = result[74]
        results2[j].append(result)

        #Progress Counter
        print(str(1+j+i*iterations) + "/" + str(iterations * len(conditions)))
        
    #Sum accuracy from all runs
    sum_results = [0] * 75
    for i in range (0, iterations):
        sum_results = np.add(sum_results, results1[i])

    #Take average of accuracy from all runs
    avg_results.append([x / iterations for x in sum_results])

#Plot validation curves of different models
plt.figure(figsize=(12, 8))
for k in range(len(conditions)):
    plt.plot(avg_results[k], label=labels[k], color=colours[k])
plt.title('Validation Accuracy by Epoch', fontsize = 25)
plt.xlabel('Epoch', fontsize = 18)
plt.ylabel('Accuracy', fontsize = 18)
plt.xticks(range(0,100,5), range(0,100,5))
plt.legend(fontsize = 18);

#Turn list into dataframe
results_df = pd.DataFrame(results2, columns=labels)
results_df.head()

#Export dataframe as CSV file
results_df.to_csv(r'/Users/jthiepler/Documents/GitHub/voice-classifier/results.csv', index=False)